<h2><strong>본격적으로 케라스와 버트를 활용하여 KorQUAD 예측 모델을 만들어 보겠습니다.</strong></h2>

텐서플로우, 판다스, 넘파이, 케라스 등 필요한 모듈들을 임포트합니다

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [24]:
!pip install numpy==1.15.4 --user

In [10]:
import json
import os
import pandas as pd
import codecs

In [22]:
from gluonnlp.data import SentencepieceTokenizer

SQUAD JSON파일을 PANDAS DATAFRAME으로 만들어주는 함수를 정의합니다. 

KorQuAD도 SQUAD랑 동일한 방식이기에, Pandas Dataframe으로 잘 변환 됩니다. 
출처 : https://www.kaggle.com/sanjay11100/squad-stanford-q-a-json-to-pandas-dataframe

In [3]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

KorQUAD 데이터를 PANDAS DATAFRAME 형식으로 로드합니다.

In [4]:
train = squad_json_to_dataframe_train("KorQuAD_v1.0_train.json")

Reading the json file
processing...


C:\Users\DeepLearning_4\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
C:\Users\DeepLearning_4\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app
C:\Users\DeepLearning_4\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


shape of the dataframe is (60407, 6)
Done


In [5]:
train

,index,question,context,text,answer_start,c_id
0,6566495-0-0,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,교향곡,54,0
1,6566495-0-1,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1악장,421,0
2,6566495-0-2,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,베토벤의 교향곡 9번,194,0
3,6566518-0-0,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트,15,0
4,6566518-0-1,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,합창교향곡,354,0
...,...,...,...,...,...,...
60402,6467478-1-1,뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",인도,73,9604
60403,6467478-2-0,균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",유전자 전달,253,9605
60404,6467478-2-1,박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",37건,129,9605
60405,6490801-2-0,NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",콜리스틴,404,9605



- bert 훈련을 위한 사전 설정을 합니다. SEQ_LEN은 문장의 최대 길이입니다. SEQ_LEN 보다 문장의 길이가 작다면 남은 부분은 0이 채워지고, 만약에 SEQ_LEN보다 문장 길이가 길다면 SEQ_LEN을 초과하는 부분이 잘리게 됩니다.  
본 문제에서는 메모리 문제 등으로 384로 정했습니다.
- BATCH_SIZE는 메모리 초과 같은 문제를 방지하기 위해 작은 수인 10으로 정했습니다. 그리고 총 훈련 에포크 수는 1로 정했습니다. 학습율(LR;Learning rate)은 3e-5로 작게 정했습니다.
- pretrained_path는 bert 사전학습 모형이 있는 폴더를 의미합니다.
- 그리고 우리가 분석할 문장이 들어있는 칼럼의 제목인 document와 긍정인지 부정인지 알려주는 칼럼을 label로 정해줍니다


In [8]:
SEQ_LEN = 384
BATCH_SIZE = 3
EPOCHS=1
LR=3e-5

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "context"
QUESTION_COLUMN = "question"
TEXT = "text"

vocab.txt에 있는 단어에 인덱스를 추가해주는 token_dict라는 딕셔너리를 생성합니다.  
우리가 분석할 문장이 토큰화가 되고, 그 다음에는 인덱스(숫자)로 변경되어서 버트 신경망에 인풋으로 들어게 됩니다.

- BERT의 토큰화는 단어를 분리하는 토큰화 방식입니다. wordpiece(단어조각?) 방식이라고 하는데, 이는 한국어를 형태소로 꼭 변환해야 할 문제를 해결해주며, 의미가 있는 단어는 밀접하게 연관이 되게 하는 장점까지 갖추고 있습니다.
- 단어의 첫 시작은 ##가 붙지 않지만, 단어에 포함되면서 단어의 시작이 아닌 부분에는 ##가 붙는 것이 특징입니다.  
- 네이버 감성분석에서 했던 것처럼, 한국어 감성분석에서는 새로 토크나이저 클래스를 상속을 받아서, 토크나이저를 재정의 해주어야 합니다.(그렇지 않으면 완전자모분리 현상 발생)

In [14]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [40]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
sp  = SentencepieceTokenizer(tokenizer)

using cached model


토큰화가 잘 되었는지 확인해 봅니다.
버트 모형은 문장 앞에 꼭 [CLS]라는 문자가 위치하고, [SEP]라는 문자가 끝에 위치합니다.  
[CLS]는 문장의 시작, [SEP]는 문장의 끝을 의미합니다.

In [42]:
print(sp("한국어 토큰화."), tok("잘되니."))

['▁한국', '어', '▁토', '큰', '화', '.'] ['▁잘', '되', '니', '▁', '.']


In [43]:
tokList = sp("한국어 모델을 공유합니다.")
tokList.insert(0, '[CLS]')
tokList.append('[SEP]')
print(tokList)

['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']


In [72]:
def token1(text):
    tokList = sp(text)
    tokList.insert(0, '[CLS]')
    tokList.append('[SEP]')
    
    return tokList

def token2(text1, text2):
    tokList1 = sp(text1)
    tokList2 = sp(text2)
    
    tokList1.insert(0, '[CLS]')
    tokList1.append('[SEP]')
    for data in tokList2:
        tokList1.append(data)
    tokList1.append('[SEP]')
    
    return tokList1

In [73]:
token2("자연어 처리가 뭐야?", "자연어 처리가 뭐야?")

['[CLS]',
 '▁자연',
 '어',
 '▁처리',
 '가',
 '▁뭐',
 '야',
 '?',
 '[SEP]',
 '▁자연',
 '어',
 '▁처리',
 '가',
 '▁뭐',
 '야',
 '?',
 '[SEP]']

In [45]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt@dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt@dl=1", field_indices=[1,2], num_discard_samples=1)

In [53]:
dir(nlp.data)

['ATISDataset',
 'BERTBasicTokenizer',
 'BERTSPTokenizer',
 'BERTSentenceTransform',
 'BERTTokenizer',
 'BakerVerb143',
 'BiggerAnalogyTestSet',
 'CR',
 'ClipSequence',
 'CoNLL2000',
 'CoNLL2001',
 'CoNLL2002',
 'CoNLL2004',
 'ConcatDataset',
 'ConstWidthBucket',
 'CorpusDataset',
 'Counter',
 'DataStream',
 'DatasetStream',
 'ExpWidthBucket',
 'Fil9',
 'FixedBucketSampler',
 'GBWStream',
 'GPT2BPEDetokenizer',
 'GPT2BPETokenizer',
 'GlueCoLA',
 'GlueMNLI',
 'GlueMRPC',
 'GlueQNLI',
 'GlueQQP',
 'GlueRTE',
 'GlueSST2',
 'GlueSTSB',
 'GlueWNLI',
 'GoogleAnalogyTestSet',
 'IMDB',
 'IWSLT2015',
 'JiebaTokenizer',
 'LinearWidthBucket',
 'MEN',
 'MPQA',
 'MR',
 'NLTKMosesDetokenizer',
 'NLTKMosesTokenizer',
 'NLTKStanfordSegmenter',
 'NumpyDataset',
 'PadSequence',
 'PrefetchingStream',
 'RadinskyMTurk',
 'RareWords',
 'SNIPSDataset',
 'SQuAD',
 'SST_1',
 'SST_2',
 'SUBJ',
 'SacreMosesDetokenizer',
 'SacreMosesTokenizer',
 'SemEval17Task2',
 'SentencepieceDetokenizer',
 'SentencepieceTokeni

In [51]:
help(dataset_train)

Help on TSVDataset in module gluonnlp.data.dataset object:

class TSVDataset(mxnet.gluon.data.dataset.SimpleDataset)
 |  Common tab separated text dataset that reads text fields based on provided sample splitter
 |  and field separator.
 |  
 |  The returned dataset includes samples, each of which can either be a list of text fields
 |  if field_separator is specified, or otherwise a single string segment produced by the
 |  sample_splitter.
 |  
 |  Example::
 |  
 |      # assume `test.tsv` contains the following content:
 |      # Id    FirstName       LastName
 |      # a     Jiheng  Jiang
 |      # b     Laoban  Zha
 |      # discard the first line and select the 0th and 2nd fields
 |      dataset = data.TSVDataset('test.tsv', num_discard_samples=1, field_indices=[0, 2])
 |      assert dataset[0] == [u'a', u'Jiang']
 |      assert dataset[1] == [u'b', u'Zha']
 |  
 |  Parameters
 |  ----------
 |  filename : str or list of str
 |      Path to the input text file or list of paths t

AttributeError: 'TSVDataset' object has no attribute 'getitem'

In [54]:
help(nlp.data.BERTSentenceTransform)

Help on class BERTSentenceTransform in module gluonnlp.data.transforms:

class BERTSentenceTransform(builtins.object)
 |  BERT style data transformation.
 |  
 |  Parameters
 |  ----------
 |  tokenizer : BERTTokenizer.
 |      Tokenizer for the sentences.
 |  max_seq_length : int.
 |      Maximum sequence length of the sentences.
 |  pad : bool, default True
 |      Whether to pad the sentences to maximum length.
 |  pair : bool, default True
 |      Whether to transform sentences or sentence pairs.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, line)
 |      Perform transformation for sequence pairs or single sequences.
 |      
 |      The transformation is processed in the following steps:
 |      - tokenize the input sequences
 |      - insert [CLS], [SEP] as necessary
 |      - generate type ids to indicate whether a token belongs to the first
 |      sequence or the second sequence.
 |      - generate valid length
 |      
 |      For sequence pairs, the input is a tuple

In [56]:
help(nlp.data.dataset)

Help on module gluonnlp.data.dataset in gluonnlp.data:

NAME
    gluonnlp.data.dataset

DESCRIPTION
    NLP Toolkit Dataset API. It allows easy and customizable loading of corpora and dataset files.
    Files can be loaded into formats that are immediately ready for training and evaluation.

CLASSES
    mxnet.gluon.data.dataset.ArrayDataset(mxnet.gluon.data.dataset.Dataset)
        NumpyDataset
    mxnet.gluon.data.dataset.Dataset(builtins.object)
        ConcatDataset
    mxnet.gluon.data.dataset.SimpleDataset(mxnet.gluon.data.dataset.Dataset)
        CorpusDataset
        TSVDataset
        TextLineDataset
    
    class ConcatDataset(mxnet.gluon.data.dataset.Dataset)
     |  Dataset that concatenates a list of datasets.
     |  
     |  Parameters
     |  ----------
     |  datasets : list
     |      List of datasets.
     |  
     |  Method resolution order:
     |      ConcatDataset
     |      mxnet.gluon.data.dataset.Dataset
     |      builtins.object
     |  
     |  Methods 

token dict의 key 값과 value 값을 바꾼 reverse_token_dict를 정의합니다.

**버트 모형에 들어갈 인풋은 토큰, 세그먼트, 포지션으로 구성됩니다.**  
버트에 인풋으로 들어가는 토큰은 문장을 토크나이징 한 후, 인덱스 번호를 매긴 것입니다.  
세그먼트는 예를 들어 문장이 두 개가 있다면, 앞의 문장과 뒤의 문장을 구분하는 것입니다.  
포지션 임베딩은 단순히 단어의 위치를 말합니다.

토큰, 세그먼트, 포지션을 인풋으로 버트 모형에 넣으면 기하학적인 문장 공간으로 임베딩이 됩니다.  
그림을 보면 my dog is cute, he likes play ##ing 두 문장이 있는데요  
KorQUAD 문제에서는 첫번째 문장이 질문, 두번째 문장이 context가 되고, 이 두 문장이 합쳐져서 하나의 문장으로 들어가게 됩니다.

![대체 텍스트](https://i.imgur.com/l9BTao3.png)

- 사전학습된 버트 모델의 인풋은 문장 토큰화가 숫자로 바뀐 것과, 앞문장인지 뒷문장인지 알려주는 문장 순서 벡터가 들어갑니다. 우리는 문장 하나를 가지고만 훈련할 것이므로 순서 벡터는 모두 0으로 통일합니다.

- 그리고 파인튜닝 시에는 문장 안에 일부 단어를 가리는 마스킹은 사용하지 않습니다.

우리가 로드하였던 KorQUAD 데이터를 **버트 모형의 입력에 맞게 변형해주는 함수**를 정의하도록 하겠습니다.

함수 내부에 tokenizer.encode 함수가 버트 모형을 토큰화해주고 토큰화 된 단어를 인덱스에 맞게 숫자로 바꿔주게 됩니다.  
**[CLS] 질문 [SEP] 문장 [SEP]** 이런 방식으로 질문과 문장이 인풋으로 들어가게 됩니다. SEQ_LEN이 384로 지정되어 있어서 길이가 384가 넘는 인풋은 문장 부분이 잘려서 들어가게 됩니다.  
SQUAD 문제에서 문장(context) 내에 text(정답)이 포함된다고 미리 말씀 드렸는데요, 길이가 384가 넘는 인풋인 경우에 context 내에 정답을 포함하고 있는 context가 잘려서 정답을 포함하지 않는 경우가 생깁니다. 이번 실습에서는 이러한 경우의 인덱스를 del_list로 지정해서, 빼도록 하겠습니다. (숙제로 남겨 두겠습니다.) 

In [74]:
question = train['question'][0]
context = train['context'][0]
text = train['text'][0]

In [75]:
# 질문
question

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

In [76]:
# 문장
context

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [77]:
# 정답
text

'교향곡'

In [78]:
print(token2(question, context))

['[CLS]', '▁바', '그', '너', '는', '▁괴', '테', '의', '▁파', '우', '스트', '를', '▁읽', '고', '▁무', '엇', '을', '▁쓰', '고', '자', '▁했', '는', '가', '?', '[SEP]', '▁18', '39', '년', '▁바', '그', '너', '는', '▁괴', '테', '의', '▁파', '우', '스트', '을', '▁처음', '▁읽', '고', '▁그', '▁내용', '에', '▁마음', '이', '▁', '끌', '려', '▁이를', '▁소재', '로', '▁해서', '▁하나', '의', '▁교', '향', '곡', '을', '▁쓰', '려는', '▁뜻을', '▁갖', '는', '다', '.', '▁이', '▁시기', '▁바', '그', '너', '는', '▁18', '38', '년에', '▁빛', '▁독', '촉', '으로', '▁산', '전', '수', '전을', '▁다', '▁', '걲', '은', '▁상황이', '라', '▁좌', '절', '과', '▁실망', '에', '▁가득', '했으며', '▁메', '피스', '토', '펠', '레스', '를', '▁만나', '는', '▁파', '우', '스트', '의', '▁심경', '에', '▁공감', '했다고', '▁한다', '.', '▁또한', '▁파', '리', '에서', '▁아', '브', '네', '크', '의', '▁지휘', '로', '▁파', '리', '▁음악', '원', '▁관', '현', '악', '단', '이', '▁연', '주', '하는', '▁베', '토', '벤', '의', '▁교', '향', '곡', '▁9', '번', '을', '▁듣고', '▁깊은', '▁감', '명을', '▁받았', '는데', ',', '▁이것', '이', '▁이', '듬', '해', '▁1', '월에', '▁파', '우', '스트', '의', '▁서', '곡', '으로', '▁쓰', '여', '진', '▁이', '▁작품', '에', '▁

우리의 목표는, 질문(question)과 문장(context)를 받아서, 정답(text)를 맞추는 모델을 만드는 것입니다.  
정답을 통째로 맞추는 것이 아니라, **토큰화된 것의 맨 앞 단어와, 맨 뒷 단어입니다.**  
토큰화된 정답은 ['[CLS]', 'saint', 'bern', '##ade', '##tte', 'sou', '##bir', '##ous', '[SEP]'] 인데, 여기서 **saint**에 해당하는 위치와 **##ous**에 해당하는 위치를 맞추는 버트 모형을 파인튜닝 하려 하는 것입니다.  

  그래서 밑에 convert_data 함수에서, 정답(text) 길이만큼 문장(context)를 슬라이딩 하면서 만약에 문장이 정답을 포함하는 위치에 도달하면, 문장에서 정답의 맨 앞이 우리가 예측할 1번째 정답, 정답의 맨 뒤가 우리가 예측할 2번째 정답이 되게 됩니다.

In [21]:
def convert_data(data_df):
    global tokenizer
    indices, segments, target_start, target_end = [], [], [], []
    for i in tqdm(range(len(data_df))):
        
        ids, segment = tokenizer.encode(data_df[QUESTION_COLUMN][i], data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        

        text = tokenizer.encode(data_df[TEXT][i])[0]

        text_slide_len = len(text[1:-1])
        for i in range(1,len(ids)-text_slide_len-1):  
            exist_flag = 0
            if text[1:-1] == ids[i:i+text_slide_len]:
              ans_start = i
              ans_end = i + text_slide_len - 1
              exist_flag = 1
              break
        
        if exist_flag == 0:
          ans_start = SEQ_LEN
          ans_end = SEQ_LEN

        indices.append(ids)
        segments.append(segment)

        target_start.append(ans_start)
        target_end.append(ans_end)

    indices_x = np.array(indices)
    segments = np.array(segments)
    target_start = np.array(target_start)
    target_end = np.array(target_end)
    
    del_list = np.where(target_start!=SEQ_LEN)[0]

    indices_x = indices_x[del_list]
    segments = segments[del_list]
    target_start = target_start[del_list]
    target_end = target_end[del_list]

    train_y_0 = keras.utils.to_categorical(target_start, num_classes=SEQ_LEN, dtype='int64')
    train_y_1 = keras.utils.to_categorical(target_end, num_classes=SEQ_LEN, dtype='int64')
    train_y_cat = [train_y_0, train_y_1]
    
    return [indices_x, segments], train_y_cat

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[QUESTION_COLUMN] = data_df[QUESTION_COLUMN].astype(str)


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [22]:
train_x, train_y = load_data(train)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60407/60407 [01:26<00:00, 702.15it/s]


In [23]:
train_x[0]

array([[   101,   9318,  78136, ...,  12178,   9011,    102],
       [   101,   9318,  78136, ...,   9011, 118783,    102],
       [   101,   9318,  78136, ...,   9011, 118783,    102],
       ...,
       [   101,   9319, 119351, ...,      0,      0,      0],
       [   101,    182,  10162, ...,      0,      0,      0],
       [   101,  19145,  17289, ...,      0,      0,      0]])

In [24]:
train.shape

(60407, 6)

In [25]:
train_x[0].shape ## 데이터 10000개 소실(데이터 10000개를 추가해주려면 별도 작업 필요)

(50609, 384)

#### 이해가 안 가실 수 있는데, 버트 인풋을 문장으로 예를 들어 만들어 보겠습니다.
#### 인풋은 총 2개가 들어갑니다
- **(토큰)** 첫번째 인풋은 토큰화 된 것이 인덱싱되어 숫자로 변환된 것  

- **(세그멘트)** 두번째 인풋은 앞문장인지 뒷문장인지 알려주는 숫자들입니다. 이번 튜토리얼에서는 파인튜닝 과정이라 앞문장 뒷문장 구분을 안하기 때문에 모두 0으로 하였습니다.  

- **(포지션)** 단어 순서에 따라서 자동으로 부여됩니다.


이해가 되셨는지요?  
구글 깃허브에서 다운받았던 사전학습된 모델을 colab으로 로드합니다.  
Training을 False로 두어서 Bert 모델에서, 마지막 트랜스포머 계층까지만 모델이 로드되게 합니다.

In [26]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=False,
    trainable=True,
    seq_len=SEQ_LEN,)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 384)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 384)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 384, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 384, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

모델의 구조를 확인합니다.  
총 12층의 트랜스포머 계층이 있음을 확인할 수 있습니다.


Transfer learning을 위해 Custom Layer를 작성해 줍니다.  
NonMasking 함수를 지정해서, Bert 모형의 자체 Masking 된 텐서들을 풀어줘야 합니다.  
이번 튜토리얼에서 만약 NonMasking 클래스를 만들지 않는다면, Bert 모형을 훈련할 수 없습니다.

In [27]:
class NonMasking(Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

Keras Custom Layer 두 개를 생성합니다.  
MyLayer_Start는 정답의 첫 번째 단어를 예측하는 것을 담당하고,  
MyLaer_End는 정답의 마지막 단어를 예측하는 것을 담당합니다.  
  
사실 두 레이어는 동일한 역할을 합니다.  
Bert 모형의 마지막 입력을 받아서, (batch_size, 384, 768)의 텐서 모양을 (batch_size, 384, 2)로 만들어주는 텐서를 곱해줍니다.  
이 다음에 i) (batch_size, 384), ii) (batch_size, 384)의 아웃풋을 출력할 수 있게 하나의 텐서를 두개로 잘라줍니다.  
  
왜 끝이 384냐면, 384개의 위치를 예측하기 때문입니다. 단어의 위치의 최대 개수는 384개로 앞서 지정하였습니다.(SEQ_LEN)


In [28]:
class MyLayer_Start(Layer):

    def __init__(self,seq_len, **kwargs):
        
        self.seq_len = seq_len
        self.supports_masking = True
        super(MyLayer_Start, self).__init__(**kwargs)

    def build(self, input_shape):
        
        self.W = self.add_weight(name='kernel', 
                                 shape=(input_shape[2],2),
                                 initializer='uniform',
                                 trainable=True)
        super(MyLayer_Start, self).build(input_shape)

    def call(self, x):
        
        x = K.reshape(x, shape=(-1,self.seq_len,K.shape(x)[2]))
        x = K.dot(x, self.W)
        
        x = K.permute_dimensions(x, (2,0,1))

        self.start_logits, self.end_logits = x[0], x[1]
        
        self.start_logits = K.softmax(self.start_logits, axis=-1)
        
        return self.start_logits

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.seq_len)


class MyLayer_End(Layer):
  def __init__(self,seq_len, **kwargs):
        
        self.seq_len = seq_len
        self.supports_masking = True
        super(MyLayer_End, self).__init__(**kwargs)
  
  def build(self, input_shape):
        
        self.W = self.add_weight(name='kernel', 
                                 shape=(input_shape[2], 2),
                                 initializer='uniform',
                                 trainable=True)
        super(MyLayer_End, self).build(input_shape)

  
  def call(self, x):

        
        x = K.reshape(x, shape=(-1,self.seq_len,K.shape(x)[2]))
        x = K.dot(x, self.W)
        x = K.permute_dimensions(x, (2,0,1))
        
        self.start_logits, self.end_logits = x[0], x[1]
        
        self.end_logits = K.softmax(self.end_logits, axis=-1)
        
        return self.end_logits

  def compute_output_shape(self, input_shape):
        return (input_shape[0], self.seq_len)

BERT 모델을 출력하는 함수를 지정합니다.  
start_answer, end_answer를 예측하게 됩니다.

In [29]:
from keras.layers import merge, dot, concatenate
from keras import metrics
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.output
  x = NonMasking()(dense)
  outputs_start = MyLayer_Start(SEQ_LEN)(x)
  outputs_end = MyLayer_End(SEQ_LEN)(x)
  bert_model = keras.models.Model(inputs, [outputs_start, outputs_end])
  bert_model.compile(
      optimizer=RAdam(learning_rate=LR, decay=0.0001),
      loss='categorical_crossentropy',
      metrics=['accuracy'])
  
  return bert_model

**모델의 FLOW를 확인해 보도록 하겠습니다.**

In [35]:
from IPython.display import SVG
from keras.utils import model_to_dot

SVG(model_to_dot(get_bert_finetuning_model(model), dpi=65).create(prog='dot', format='svg'))

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

훈련을 시작합니다.  
1epoch을 훈련해 보고, 결과를 확인하고 훈련을 다시 시작할 예정입니다.

In [30]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
bert_model.summary()
history = bert_model.fit(train_x, train_y, batch_size=BATCH_SIZE, validation_split=0.05, shuffle=False, verbose=1)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 384)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 384)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 384, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 384, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

Train on 48078 samples, validate on 2531 samples
Epoch 1/1
48078/48078 [==============================] - 4372s 91ms/step - loss: 1.7078 - my_layer__start_1_loss: 0.8058 - my_layer__end_1_loss: 0.9021 - my_layer__start_1_accuracy: 0.7764 - my_layer__end_1_accuracy: 0.7571 - val_loss: 1.1800 - val_my_layer__start_1_loss: 0.5452 - val_my_layer__end_1_loss: 0.6344 - val_my_layer__start_1_accuracy: 0.8435 - val_my_layer__end_1_accuracy: 0.8202


In [ ]:
bert_model = get_bert_finetuning_model(model)
bert_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 384)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 384)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 384, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 384, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

BERT MODEL을 저장합니다.

In [31]:
bert_model.save_weights("korquad_wordpiece.h5")

버트 모형을 다시 훈련합니다.  
이번에는 validation_split을 입력하지 않아서 전체 데이터가 훈련 되도록 만들어 줍니다.

In [45]:
bert_model.compile(optimizer=RAdam(learning_rate=0.00003, decay=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
bert_model.fit(train_x, train_y, batch_size=BATCH_SIZE, shuffle=False, verbose=1)

Epoch 1/1
50609/50609 [==============================] - 4605s 91ms/step - loss: 0.9107 - my_layer__start_1_loss: 0.4158 - my_layer__end_1_loss: 0.4949 - my_layer__start_1_accuracy: 0.8747 - my_layer__end_1_accuracy: 0.8588


In [46]:
bert_model.save_weights("korquad_wordpiece_2.h5")

버트 모형을 한번 더 훈련시켜 줍니다. learning_rate을 0.00001로 바꿔 줍니다.

In [47]:
bert_model.compile(optimizer=RAdam(learning_rate=0.00001, decay=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
bert_model.save_weights("korquad_wordpiece_3.h5")

재사용을 위해 bert_model을 지드라이브에 저장해줍니다.

버트 모형을 로드해줍니다. 이미 로드하였던 모델에 계수들만 살짝 얹혀 줍니다.

In [49]:
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights("korquad_wordpiece_3.h5")

Test data set에 대한 bert_input을 만들어 줍니다.  
Train data set과는 다르게 label을 생성하지 않습니다.

In [32]:
def convert_pred_data(question, doc):
    global tokenizer
    indices, segments = [], []
    ids, segment = tokenizer.encode(question, doc, max_len=SEQ_LEN)
    indices.append(ids)
    segments.append(segment)
    indices_x = np.array(indices)
    segments = np.array(segments)
    return [indices_x, segments]

def load_pred_data(question, doc):
    data_x = convert_pred_data(question, doc)
    return data_x

질문과 문장을 받아 답을 알려주는 함수를 정의합니다.

In [33]:
def predict_letter(question, doc):
  
  test_input = load_pred_data(question, doc)
  test_start, test_end = bert_model.predict(test_input)
  
  indexes = tokenizer.encode(question, doc, max_len=SEQ_LEN)[0]
  start = np.argmax(test_start, axis=1).item()
  end = np.argmax(test_end, axis=1).item()
  start_tok = indexes[start]
  end_tok = indexes[end]
  print("Question : ", question)
  
  print("-"*50)
  print("Context : ", end = " ")
  
  def split_text(text, n):
    for line in text.splitlines():
        while len(line) > n:
           x, line = line[:n], line[n:]
           yield x
        yield line

  

  for line in split_text(doc, 150):
    print(line)

  print("-"*50)
  print("ANSWER : ", end = " ")
  print("\n")
  sentences = []
  
  for i in range(start, end+1):
    token_based_word = reverse_token_dict[indexes[i]]
    sentences.append(token_based_word)
    print(token_based_word, end= " ")
  
  print("\n")
  print("Untokenized Answer : ", end = "")
  for w in sentences:
    if w.startswith("##"):
      w = w.replace("##", "")
    else:
      w = " " + w
    
    print(w, end="")
  print("")

KorQUAD 데이터 셋에서 test 용도로 쓰이는 dev 파일을 PANDAS DATAFRAME 형식으로 불러오는 함수를 정의합니다.  
train 데이터와 모양이 약간 다르기 때문에, 함수를 새로 정의해야 합니다.

In [34]:
def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = idx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [35]:
input_file_path = 'KorQuAD_v1.0_dev.json'
record_path = ['data','paragraphs','qas','answers']
verbose = 0
dev = squad_json_to_dataframe_dev(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (5774, 5)
Done


TEST DATA가 잘 불려왔는지 확인해 보겠습니다.

In [58]:
dev

,id,question,context,answers,c_id
0,6548850-0-0,임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,"[{'text': '1989년 2월 15일', 'answer_start': 0}]",0
1,6548850-0-1,1989년 6월 30일 평양축전에 대표로 파견 된 인물은?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,"[{'text': '임수경', 'answer_start': 125}]",0
2,6548853-0-0,임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,"[{'text': '1989년', 'answer_start': 0}]",0
3,6548853-0-1,임종석을 검거한 장소는 경희대 내 어디인가?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,"[{'text': '학생회관 건물 계단', 'answer_start': 365}]",0
4,6548853-0-2,임종석이 조사를 받은 뒤 인계된 곳은 어딘가?,1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률...,"[{'text': '서울지방경찰청 공안분실', 'answer_start': 457}]",0
...,...,...,...,...,...
5769,6511152-4-0,중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생한 년도는?,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,"[{'text': '2016년', 'answer_start': 0}]",959
5770,6511152-4-1,갤럭시 S7엣지를 만든 회사는?,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,"[{'text': '삼성전자', 'answer_start': 83}]",959
5771,6511152-4-2,2016년 9월 4일 갤럭시 S7엣지가 폭발한 사건이 발생한 지역은?,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,"[{'text': '중동 지역', 'answer_start': 12}]",959
5772,6535617-4-0,갤럭시 노트 7은 출시 며칠만에 기기 결함으로 터지기 시작하였나?,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,"[{'text': '18일', 'answer_start': 286}]",959


테스트 데이터에 대해서 결과를 확인합니다.  
훈련에 사용하지 않은 테스트 데이터에 대한 예측을 제법 잘 수행하는 것을 보실 수 있겠습니다.

In [57]:
import random
for i in random.sample(range(100),100):
  doc = dev['context'][i]
  question = dev['question'][i]
  answers = dev['answers'][i]
  predict_letter(question, doc)
  print("")
  print("real answer : ", answers)
  print("")

Question :  반류마루가 미야코 만 해전에서 폭풍우를 만나 대기하고 있던 항구의 이름은 무엇인가?
--------------------------------------------------
Context :  일련의 하코다테 전쟁은 적아 쌍방의 문서에 마쓰오카 바키치 함장의 능란한 조함 능력과 냉정한 지휘만이 기록되어 있다. 함포 사격으로 마쓰마에 성을 공격하여 엄호한 이후, 1869년 메이지 2년 3월 25일 미야코 만 해전에서는 폭풍우를 만나 요함과 헤어졌을 때에 만날 
약속했던 하치노헤 항에서 대기하고 있었기 때문에 참전에는 이르지 못했다. 이 폭풍우 때도 “함장 마쓰오카 바키치는 배를 조정하는 명수로 로프 하나 손상되지 않았다”고 타고 있던 하야시 다다스가 남긴 바 있다. 이 귀로에서 신정부 군의 철갑함의 추격을 받았다. 기관 능력
의 차이로 인한 속도차 때문에 도주가 불가능하다고 판단하고 맞장 공격을 하겠다고 전투 준비를 했지만, 철갑선의 사정거리에 들어간 순간에 순풍이 불기 시작하여 추격을 뿌리치고 하코다테로 돌아올 수 있었다.
--------------------------------------------------
ANSWER :  

하 ##치 ##노 ##헤 항 

Untokenized Answer :  하치노헤 항

real answer :  [{'text': '하치노헤', 'answer_start': 155}]

Question :  노아의 방주에 대해 기록하고있는 복음서는 무엇인가?
--------------------------------------------------
Context :  노아는 하나님의 명령에 따라 배를 만들고 가족과 정결한 짐승 암수 일곱 마리씩, 부정한 짐승 암수 한 마리씩(혹은 두 마리씩; 사본에 따라 다름), 그리고 새 암수 일곱 마리씩을 싣고 밀어닥친 홍수를 피하였다. 모든 사람들이 타락한 생활에 빠져 있어 하나님이 홍수로 심
판하려 할 때 홀로 바르게 살던 노아는 하나님의 특별한 계시로 홍수가 올 것

Question :  역사학과 과학의 발달로 홍수지질학은 어떤 과학으로 남게 되었는가?
--------------------------------------------------
Context :  역사학과 과학의 발달이 더뎠던 고대사회에서는, 성경이 단순한 교리적인 부분 뿐 아니라 역사책으로서의 권위도 높았기에 노아의 방주를 역사적인 존재로서 다루고 있었다. 이는 제칠일안식교에서 비롯된 의사과학의 한 종류인 유사지질학인 홍수지질학과 같은 것에 영향을 주었으며, 
과거 신학에서는 이러한 근본주의적 해석을 받아들여 역사와 사회적인 모든 부분에 있어 성경을 교과서로 채택할 것을 촉구했다. 이러한 홍수지질학을 주장했던 유사지질학자들은 성경에 나오는 노아의 홍수가 어딘가에 그 흔적이 남아 있을것이라고 주장하며 노아의 방주를 찾기 위한 
노력을 했다고 주장한다. 이들은 같은 메소포타미아 지방의 신화인 이슬람교 경전이나 길가메쉬 서사시등의 신화를 들어서 이를 근거라고 주장하기도 했다. 그러나 이러한 전통적 근본주의적 시각은 과거에는 상당히 힘을 얻었으나, 역사학과 과학의 발달에 따라 힘을 잃게 되었고, 
홍수지질학은 유사과학으로서 남게 되었다. 현대에는 뒤의 실존논란에서 다루는 것처럼 이러한 근본주의적 해석은 비과학적인 해석으로 여기는 것이 일반적이지만, 남침례교로 대표되는 극보수주의계열 기독교에서는 아직도 이것이 받아들여지고 있다.
--------------------------------------------------
ANSWER :  

유 ##사 ##과 ##학 

Untokenized Answer :  유사과학

real answer :  [{'text': '유사과학', 'answer_start': 457}]

Question :  현대에 노아의 방주에 대학 근본주의적 해석은 어떻게 여겨지는가?
--------------------------------------------------
Context :  역사학과 과학의 발달이 더뎠던 고대사회에서는, 성경이 단순

1869 ##년 메 ##이지 2년 3월 25일 

Untokenized Answer :  1869년 메이지 2년 3월 25일

real answer :  [{'text': '1869년 메이지 2년 3월 25일', 'answer_start': 95}]

Question :  헤이그가 군에서 퇴역한 년도는 몇년도입니까?
--------------------------------------------------
Context :  헤이그는 닉슨 대통령이 그를 사성 장군과 육군 부참모로 진급시킬 때 집중 광선과 논쟁으로 들어갔다. 헤이그를 군사의 최상으로 밀어넣은 닉슨의 행동은 대통령의 남자들을 다양한 연방 대리법에서 권한의 직우들로 놓은 노력과 함께 일치였다. 하지만 그는 곧 백악관으로 돌아가 
1973년부터 1974년까지 대통령 특별 보좌관을 지냈다. 워터게이트 사건이 일어난지 한달 후, 헤이그는 포위된 닉슨 대통령을 위한 치명적 역할을 하였다. 그일은 8월 닉슨의 사임과 제럴드 포드의 대통령으로 계승으로 이끈 협상들에서 헤이그가 수단이었던 우연이 아니었다.
 곧 후에 헤이그는 미국 유럽 연합군 최고사령부의 최고 사령관으로 임명되었다. 그는 나토에서 다음 5년을 보내고 1979년 군에서 퇴역하여 미국 기술 주식 회사의 우두머리가 되었다.
--------------------------------------------------
ANSWER :  

1979 ##년 

Untokenized Answer :  1979년

real answer :  [{'text': '1979년', 'answer_start': 363}]

Question :  헤이그를 기용한 국제적 상담 회사의 이름은 무엇입니까?
--------------------------------------------------
Context :  그의 편에 헤이그는 지구촌의 논점들의 국내적 정치 노력들에 관해서만 근심한 레이건의 가까운 조언자들을 "외교 정책의 아마추어"로 묘사하였다. 1982년 6월 25일

Question :  미국 군대에서 두번째로 높은 직위는?
--------------------------------------------------
Context :  알렉산더 메이그스 헤이그 2세(영어: Alexander Meigs Haig, Jr., 1924년 12월 2일 ~ 2010년 2월 20일)는 미국의 국무 장관을 지낸 미국의 군인, 관료 및 정치인이다. 로널드 레이건 대통령 밑에서 국무장관을 지냈으며, 리처드 닉슨과 제럴
드 포드 대통령 밑에서 백악관 비서실장을 지냈다. 또한 그는 미국 군대에서 2번째로 높은 직위인 미국 육군 부참모 총장과 나토 및 미국 군대의 유럽연합군 최고사령관이었다. 한국 전쟁 시절 더글러스 맥아더 유엔군 사령관의 참모로 직접 참전하였으며, 로널드 레이건 정부 출
범당시 초대 국무장관직을 맡아 1980년대 대한민국과 미국의 관계를 조율해 왔다. 저서로 회고록 《경고:현실주의, 레이건과 외교 정책》(1984년 발간)이 있다.
--------------------------------------------------
ANSWER :  

미국 육 ##군 부 ##참 ##모 총 ##장 

Untokenized Answer :  미국 육군 부참모 총장

real answer :  [{'text': '미국 육군 부참모 총장', 'answer_start': 204}]

Question :  노아의 방주는 총 몇층으로 되어 있었는가?
--------------------------------------------------
Context :  노아는 하나님의 명령에 따라 배를 만들고 가족과 정결한 짐승 암수 일곱 마리씩, 부정한 짐승 암수 한 마리씩(혹은 두 마리씩; 사본에 따라 다름), 그리고 새 암수 일곱 마리씩을 싣고 밀어닥친 홍수를 피하였다. 모든 사람들이 타락한 생활에 빠져 있어 하나님이 홍수로 심
판하려 할 때 홀로 바르게 살던 노아는 하나님의 특별한 계시로 홍수가 올 것을 미리 알게 된다. 그는 길이 300 규빗, 너비 50

Question :  목재로 만들어진 선박은 강도 상의 통상 길이 몇m가 한계인가?
--------------------------------------------------
Context :  창조과학회에서는 또한 노아의 방주가 안정적인 구조였다고 주장하지만, 이와는 달리 노아의 방주는 항해가 불가능한 설계에 가깝다. 실제로 창조과학에서 주장하는 방주의 크기와 철제 부품을 사용하지 않은 목재 선박 중에서 가장 큰 수준의 선박들을 비교하면 배수량이 두배 이상 
차이난다. 그리고 목재 선박은 강도 상의 문제 때문에 통상 길이 100m, 배수량 2000톤 정도가 한계로 여겨져 왔다. 창조과학회에서는 노아의 방주의 안정성을 실험하기 위한 연구가 있다고 주장하기도 하나, 그 자체의 불합리성에 대한 비판을 받고 있으며, 관련 주요 연
구자는 지질학 석사학위, 생물학 학사학위를 가진 초등학교 교사로서, 주류 학계의 학회나 저널 등에 발표한 적이 없으며 또한 정당한 피어 리뷰에 의해 검증받지 않았다.
--------------------------------------------------
ANSWER :  

100m 

Untokenized Answer :  100m

real answer :  [{'text': '100m', 'answer_start': 186}]

Question :  1950년 헤이그와 결혼한 여자의 이름은?
--------------------------------------------------
Context :  노터데임 대학교에서 2년간 합리적으로 심각한 공부를 한 후 헤이그는 1944년 미국 육군사관학교로 임명을 획득하여 자신의 어린 시절을 군사 경력의 야망으로 알아챘다. 그 경력은 헤이그의 학문적 경연이 암시하려고 한것보다 더욱 극적이었으며 그는 1947년 310의 동기병
에서 217번째 사관으로서 졸업하였다. 22세의 소위로 헤이그는 처음에 캔자스 주 포트라일리에서 정통 제병 연합부대로, 그러고나서 켄터키 주 포트녹스에 있는 기갑 훈련

퍼 ##트 ##리 ##샤 앤 ##토 ##이 ##넷 폭 ##스 

Untokenized Answer :  퍼트리샤 앤토이넷 폭스

real answer :  [{'text': '퍼트리샤 앤토이넷 폭스', 'answer_start': 345}]

Question :  임종석을 검거한 장소는 경희대 내 어디인가?
--------------------------------------------------
Context :  1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가
 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대
학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서
울 장안동의 서울지방경찰청 공안분실로 인계되었다.
--------------------------------------------------
ANSWER :  

경 ##희 ##대학교 학 ##생 ##회 ##관 

Untokenized Answer :  경희대학교 학생회관

real answer :  [{'text': '학생회관 건물 계단', 'answer_start': 365}]

Question :  1989년 6월 30일 평양축전에 대표로 파견 된 인물은?
--------------------------------------------------
C

상 ##징 ##적 의 ##미 

Untokenized Answer :  상징적 의미

real answer :  [{'text': '상징적 의미', 'answer_start': 329}]

Question :  1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?
--------------------------------------------------
Context :  1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가
 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대
학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서
울 장안동의 서울지방경찰청 공안분실로 인계되었다.
--------------------------------------------------
ANSWER :  



Untokenized Answer : 

real answer :  [{'text': '임종석', 'answer_start': 87}]

Question :  노아의 방주를 만든 재질은?
--------------------------------------------------
Context :  노아는 하나님의 명령에 따라 배를 만들고 가족과 정결한 짐승 암수 일곱 마리씩, 부정한 짐승 암수 한 

남 ##침 ##례 ##교 

Untokenized Answer :  남침례교

real answer :  [{'text': '남침례교', 'answer_start': 536}]

Question :  역사학과 과학의 발달이 미비했을 때 전통 신학계에서는 어떠한 시작으로 노아의 방주를 역사적 사실로 기술하였는가?
--------------------------------------------------
Context :  역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아
의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가
톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다
--------------------------------------------------
ANSWER :  

근 ##본 ##주의 ##적 시 ##각 

Untokenized Answer :  근본주의적 시각

real answer :  [{'text': '근본주의적', 'answer_start': 31}]

Question :  노아의 방주 안전성을 연구하는 주요 연구자의 직업은?
--------------------------------------------------
Context :  창조과학회에서는 또한 노아의 방주가 안정적인 구조였다고 주장하지만, 이와는 달리 노아의 방주는 항해가 불가능한 설계에 가깝다. 실제로 창조과학에서 주장하는 방

real answer :  [{'text': '중국', 'answer_start': 555}]

Question :  하코다테 전쟁 시 반류마루의 함장의 이름은 무엇인가?
--------------------------------------------------
Context :  일련의 하코다테 전쟁은 적아 쌍방의 문서에 마쓰오카 바키치 함장의 능란한 조함 능력과 냉정한 지휘만이 기록되어 있다. 함포 사격으로 마쓰마에 성을 공격하여 엄호한 이후, 1869년 메이지 2년 3월 25일 미야코 만 해전에서는 폭풍우를 만나 요함과 헤어졌을 때에 만날 
약속했던 하치노헤 항에서 대기하고 있었기 때문에 참전에는 이르지 못했다. 이 폭풍우 때도 “함장 마쓰오카 바키치는 배를 조정하는 명수로 로프 하나 손상되지 않았다”고 타고 있던 하야시 다다스가 남긴 바 있다. 이 귀로에서 신정부 군의 철갑함의 추격을 받았다. 기관 능력
의 차이로 인한 속도차 때문에 도주가 불가능하다고 판단하고 맞장 공격을 하겠다고 전투 준비를 했지만, 철갑선의 사정거리에 들어간 순간에 순풍이 불기 시작하여 추격을 뿌리치고 하코다테로 돌아올 수 있었다.
--------------------------------------------------
ANSWER :  

마 ##쓰 ##오 ##카 바 ##키 ##치 

Untokenized Answer :  마쓰오카 바키치

real answer :  [{'text': '마쓰오카 바키치', 'answer_start': 24}]

Question :  함장 마쓰오카 바키치는 배를 조정하는 명수로 로프 하나 손상되지 않았다고 말한 사람은?
--------------------------------------------------
Context :  일련의 하코다테 전쟁은 적아 쌍방의 문서에 마쓰오카 바키치 함장의 능란한 조함 능력과 냉정한 지휘만이 기록되어 있다. 함포 사격으로 마쓰마에 성을 공격하여 엄호한 이후, 1869년 메이지 2년 3월 25일 미야코 만 해전

한번 만들어진 BERT 모형에 질문을 해 볼까요?  
나무위키에서 데이터를 가져와서 질문을 해보겠습니다.

In [63]:
doc = "2000년 5월 소리바다에서 개발한 mp3 음악파일 교환 서비스로 P2P 프로그램으로 출발해 대표적인 음악파일 교환 서비스로 성장하였다. 저작권 침해 문제로 서비스 중지 가처분 결정을 받았으나 소리바다5로 2006년 7월 전면 유료화 서비스로 전환하였고 월정액제와 자유이용권 등의 제도를 운영하고 있다.사용자끼리 서로의 mp3 파일을 검색하고다운로드받을 수 있는 개인 대 개인(P2P) 프로그램으로 출발하였다. 2000년 5월부터 온라인 서비스에 들어가 국내 대표적인음악파일 교환 서비스로 자리잡았다. 그러나 한국음반산업협회에서 소리바다를 상대로가처분신청을 제기하였고, 2002년 7월 11일법원은 소리바다의 음악파일 공유 서비스가저작권을 침해할 소지가 있다고 판단해 서비스 중지 가처분 결정을 내려 7월 31일 이후 서비스가 중단됐다.상황이 이렇게 되자 8월 24일 중앙집중식 검색 기능을 없앤 새 파일 교환 프로그램 소리바다 2를 개발하였다. 이 프로그램은 메인 서버 없이 슈퍼피어(SuperPeer) 방식으로 사용자 리스트를 받을 수 있으며, 전반적으로 사용자 인터페이스를 개선한 것이었다.소리바다는 2003년 11월 주식회사로 법인 전환을 하였다. 2004년 7월 '소리바다3'를 출시하였으며, 12월 유료 mp3＃을 열었다. 2005년 11월 서비스를 중단하였다가 2006년 3월 '소리바다5'라는 이름으로 서비스를 재개하였으며, 7월부터 전면 유료화되었다. '소리바다5'는 회원들이 공유한 mp3파일을 실시간으로 검색하여 원하는 파일을 다운로드할 수 있는 P2P 프로그램으로 운영되며, 월정액제의 자유이용권을 구매하여 이용할 수 있다. 다운로드하면서 미리듣기를 할 수 있고, 원하는 mp3파일의 가사도 볼 수 있다. 또 오르골이라는 무제한 용량의 저장공간도 제공된다."
question = "mp3는 언제 개발 되었어?"

predict_letter(question, doc)

Question :  mp3는 언제 개발 되었어?
--------------------------------------------------
Context :  2000년 5월 소리바다에서 개발한 mp3 음악파일 교환 서비스로 P2P 프로그램으로 출발해 대표적인 음악파일 교환 서비스로 성장하였다. 저작권 침해 문제로 서비스 중지 가처분 결정을 받았으나 소리바다5로 2006년 7월 전면 유료화 서비스로 전환하였고 월정액제와 자유
이용권 등의 제도를 운영하고 있다.사용자끼리 서로의 mp3 파일을 검색하고다운로드받을 수 있는 개인 대 개인(P2P) 프로그램으로 출발하였다. 2000년 5월부터 온라인 서비스에 들어가 국내 대표적인음악파일 교환 서비스로 자리잡았다. 그러나 한국음반산업협회에서 소리바다
를 상대로가처분신청을 제기하였고, 2002년 7월 11일법원은 소리바다의 음악파일 공유 서비스가저작권을 침해할 소지가 있다고 판단해 서비스 중지 가처분 결정을 내려 7월 31일 이후 서비스가 중단됐다.상황이 이렇게 되자 8월 24일 중앙집중식 검색 기능을 없앤 새 파일
 교환 프로그램 소리바다 2를 개발하였다. 이 프로그램은 메인 서버 없이 슈퍼피어(SuperPeer) 방식으로 사용자 리스트를 받을 수 있으며, 전반적으로 사용자 인터페이스를 개선한 것이었다.소리바다는 2003년 11월 주식회사로 법인 전환을 하였다. 2004년 7월 
'소리바다3'를 출시하였으며, 12월 유료 mp3＃을 열었다. 2005년 11월 서비스를 중단하였다가 2006년 3월 '소리바다5'라는 이름으로 서비스를 재개하였으며, 7월부터 전면 유료화되었다. '소리바다5'는 회원들이 공유한 mp3파일을 실시간으로 검색하여 원하는 
파일을 다운로드할 수 있는 P2P 프로그램으로 운영되며, 월정액제의 자유이용권을 구매하여 이용할 수 있다. 다운로드하면서 미리듣기를 할 수 있고, 원하는 mp3파일의 가사도 볼 수 있다. 또 오르골이라는 무제한 용량의 저장공간도 제공된다.
---------------------

In [51]:
doc = "태양계의 5번째 행성이며, 태양계의 행성 중 가장 부피가 크고 무겁다. 반지름은 지구의 11.2배, 부피는 지구의 1300배가 넘는다. 질량은 지구의 318배다. 부피에 비해 질량이 작은 이유는 암석형 행성보다 밀도가 낮은 성분들이 주요 구성성분인 가스형 행성이기 때문이다. 그럼에도 목성의 질량은 다른 태양계 행성들을 합친 것보다도 무겁다. 심지어 그 7개 행성의 질량을 몽땅 다 합쳐도 목성의 절반도 되지 않는다. 태양계에서 태양이 99.86%를 차지하고, 목성은 나머지 0.14% 중에서 약 2/3인 0.095%를 차지한다. 뒤를 이어 토성이 0.029%를 차지하며, 나머지 행성들을 모두 합쳐도 태양계 질량의 0.016% 정도 밖에 되지 않는다."
question = "목성의 부피는 지구의 몇배인가?"

predict_letter(question, doc)

Question :  목성의 부피는 지구의 몇배인가?
--------------------------------------------------
Context :  태양계의 5번째 행성이며, 태양계의 행성 중 가장 부피가 크고 무겁다. 반지름은 지구의 11.2배, 부피는 지구의 1300배가 넘는다. 질량은 지구의 318배다. 부피에 비해 질량이 작은 이유는 암석형 행성보다 밀도가 낮은 성분들이 주요 구성성분인 가스형 행성이기 때문
이다. 그럼에도 목성의 질량은 다른 태양계 행성들을 합친 것보다도 무겁다. 심지어 그 7개 행성의 질량을 몽땅 다 합쳐도 목성의 절반도 되지 않는다. 태양계에서 태양이 99.86%를 차지하고, 목성은 나머지 0.14% 중에서 약 2/3인 0.095%를 차지한다. 뒤를 
이어 토성이 0.029%를 차지하며, 나머지 행성들을 모두 합쳐도 태양계 질량의 0.016% 정도 밖에 되지 않는다.
--------------------------------------------------
ANSWER :  

1300 ##배 

Untokenized Answer :  1300배


In [56]:

doc = "사스와 메르스처럼 코로나 바이러스의 보고되지 않은 종에 인한 감염으로 발생하는 호흡기 질환이다. 최초 발생 원인과 전파 경로는 아직 정확히 밝혀지지 않았다.발병 초기에 보고된 불상의 폐병 증상으로 대중적으로는 '우한 폐렴' 등의 키워드로 불렸으며, 현재 세계보건기구(WHO)에서는 Novel coronavirus(2019-nCoV)라는 표현을 사용 중이다(#). 미국, 영국 등 일부 외신에서는 Wuhan coronavirus라는 표현을 사용하기도 한다. 현재 미국 정부는 질병 명칭에는 Wuhan을 넣지 않은 Coronavirus라고만 지칭 중이다.[11] 대한민국 정부에서 잠정적으로 사용하는 질병 명칭은 신종 코로나바이러스감염증이다. 이는 2015년부터 낙인 효과를 우려한 WHO에서 병명에 지역 이름을 넣는 것을 피하도록 한 권고[12](#)에 따른 것이다. 물론 해당 권고는 구속력이 없으며, 정부나 공공 의료기관이 아닌 일반인, 그리고 언론이 우한 폐렴을 사용하는 것에도 문제가 없다. 실제로 외신이나 당사자인 중국 본토에서도 '우한' 혹은 '중국' 명칭을 사용하고 있다. #기사 사태 초반에는 우한시 안에 국한될 것으로 판단하는 이들이 많았으나, 점차 우한 외 후베이성과 인근 지역, 그리고 중국을 벗어나 해외로까지 퍼지면서 상황이 심각해졌다. 우한시의 인구는 약 1,100만 명이며, 하필 맞물린 춘절[13]로 인한 인구 대이동으로 병이 사방으로 퍼질 위험이 있어 중국 현지에 비상이 걸렸다. 게다가 춘절 기간 중 중국인 관광객들이 해외여행을 많이 가다보니 세계 여러 나라에서도 촉각이 곤두서있다. 따라서 춘절 전후가 감염병 확산의 고비라 할 수 있다."
question = "우한 폐렴은 어디서 발생하였는가?"

predict_letter(question, doc)

Question :  우한 폐렴은 어디서 발생하였는가?
--------------------------------------------------
Context :  사스와 메르스처럼 코로나 바이러스의 보고되지 않은 종에 인한 감염으로 발생하는 호흡기 질환이다. 최초 발생 원인과 전파 경로는 아직 정확히 밝혀지지 않았다.발병 초기에 보고된 불상의 폐병 증상으로 대중적으로는 '우한 폐렴' 등의 키워드로 불렸으며, 현재 세계보건기구(W
HO)에서는 Novel coronavirus(2019-nCoV)라는 표현을 사용 중이다(#). 미국, 영국 등 일부 외신에서는 Wuhan coronavirus라는 표현을 사용하기도 한다. 현재 미국 정부는 질병 명칭에는 Wuhan을 넣지 않은 Coronavirus라고만
 지칭 중이다.[11] 대한민국 정부에서 잠정적으로 사용하는 질병 명칭은 신종 코로나바이러스감염증이다. 이는 2015년부터 낙인 효과를 우려한 WHO에서 병명에 지역 이름을 넣는 것을 피하도록 한 권고[12](#)에 따른 것이다. 물론 해당 권고는 구속력이 없으며, 정
부나 공공 의료기관이 아닌 일반인, 그리고 언론이 우한 폐렴을 사용하는 것에도 문제가 없다. 실제로 외신이나 당사자인 중국 본토에서도 '우한' 혹은 '중국' 명칭을 사용하고 있다. #기사 사태 초반에는 우한시 안에 국한될 것으로 판단하는 이들이 많았으나, 점차 우한 외
 후베이성과 인근 지역, 그리고 중국을 벗어나 해외로까지 퍼지면서 상황이 심각해졌다. 우한시의 인구는 약 1,100만 명이며, 하필 맞물린 춘절[13]로 인한 인구 대이동으로 병이 사방으로 퍼질 위험이 있어 중국 현지에 비상이 걸렸다. 게다가 춘절 기간 중 중국인 관광
객들이 해외여행을 많이 가다보니 세계 여러 나라에서도 촉각이 곤두서있다. 따라서 춘절 전후가 감염병 확산의 고비라 할 수 있다.
--------------------------------------------------
ANSWER :  



Untokenized Ans